# Api URLS
## https://www.alphavantage.co/documentation/
# Feel free to add your keys to config.py.  We can import them as needed in the notebook as variables

In [9]:
# import dependencies
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Config file keys.  We can import multiple keys variales in here
from config import av_key # api key for alpha vantage

In [ ]:
# pretty print a json
print(json.dumps("INSERJSONHERE", indent=4, sort_keys=True))

In [ ]:
url_alpha_vantage = "https://www.alphavantage.co" 


url_response = requests.get(url).json()